# micrograd exercises

Exercises from the [building micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0).<br>
This notebook is a slightly re-formatted version of [the original micrograd exercise notebook](https://colab.research.google.com/drive/1FPTx1RXtBfc4MaTkf7viZZD4U2F9gtKN?usp=sharing).

1. Watch the [building micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. Come back and complete these exercises to level up :)

## Section 1: Derivatives

Here is a mathematical expression that takes 3 inputs and produces one output:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from math import sin, cos, exp, log

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

Write the function $df$ that returns the analytical gradient of $f$ i.e. use your skills from calculus to take the derivative, then implement the formula.<br>
If you do not calculus, feel free to ask wolframalpha, e.g.: https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

In [ ]:
def gradf(a, b, c):
  # --------------------
  # TODO: Your code here to return [df/da, df/db, df/dc]
  # --------------------
  return [0, 0, 0]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

Now estimate the gradient numerically without any calculus, using the approximation we used in the video.<br>
You should not call the function `df` from the last cell.

In [ ]:
# -----------
# TODO: Your code here
numerical_grad = [0, 0, 0]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

There is an alternative formula that provides a much better numerical approximation to the derivative of a function.<br>
Learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative

**Implement it.**<br><br>
Confirm that for the *same step size $h$* this version gives a better approximation.

In [ ]:
# -----------
# TODO: Your code here
numerical_grad2 = [0, 0, 0]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

## Section 2: Support for Softmax

Without referencing the code/video __too__ much, make the following two cells work.

You'll have to implement (in some cases re-implemented) a number of functions of the Value object, similar to what we've seen in the video.<br>
Instead of the squared error loss, this implements the __negative log likelihood loss__, which is very often used in classification.

In [ ]:
# Value class starter code, with many functions taken out
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # TODO: Your code here
  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:
# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

Verify: The gradient using the library `torch` should be exactly the same.

In [ ]:
import torch

# TODO: Your code here

<b>Congrats! You leveled up!</b>